# 基于T5的文本摘要

## Step1 导入相关包

In [ ]:
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

## Step2 加载数据集

## Step3 数据处理

## Step4 创建模型

## Step5 创建评估函数

## Step6 配置训练参数

## Step7 创建训练器

## Step8 模型训练

## Step9 模型推理